A data analysis of the modern historical success and failures of the Miami Hurricanes.

In [12]:
--Label all home wins and loses
WITH home AS(
	SELECT 
		game_id,
		CASE WHEN home_points::integer > away_points::integer THEN 'win'
		ELSE 'loss' end as home_record
	FROM 'cfb_games_in_data_repo.csv'
	WHERE home_team = 'Miami'),
--Label all away wins and loses
away AS(
SELECT 
		game_id,
		CASE WHEN home_points::integer < away_points::integer THEN 'win'
		ELSE 'loss' end as away_record
	FROM 'cfb_games_in_data_repo.csv'
	WHERE away_team = 'Miami')
-- Select all info about every miami game in data base
SELECT
	g.season,
	g.home_team,
	g.away_team,
	home.home_record,
	away.away_record
FROM 'cfb_games_in_data_repo.csv' as g
LEFT JOIN away
	ON g.game_id = away.game_id
LEFT JOIN home
	ON home.game_id = g.game_id
WHERE (g.home_team = 'Miami'
	OR g.away_team = 'Miami') 
	
ORDER BY start_date
;

,season,home_team,away_team,home_record,away_record
0,2003,Louisiana Tech,Miami,None,win
1,2003,Miami,Florida,win,None
2,2003,Miami,East Carolina,win,None
3,2003,Boston College,Miami,None,win
4,2003,Miami,West Virginia,win,None
...,...,...,...,...,...
218,2020,NC State,Miami,None,win
219,2020,Virginia Tech,Miami,None,win
220,2020,Duke,Miami,None,win
221,2020,Miami,North Carolina,loss,None


In [2]:
-- Lets look at record by season
SELECT 
	season,
	sum(CASE WHEN home_team = 'Miami' AND home_points::integer > away_points::integer THEN 1
	   WHEN away_team = 'Miami' AND away_points::integer > home_points::integer THEN 1
	   ELSE 0 end) as wins,
	sum(CASE WHEN home_team = 'Miami' AND home_points::integer < away_points::integer THEN 1
	   WHEN away_team = 'Miami' AND away_points::integer < home_points::integer THEN 1
	   ELSE 0 end) as loses
FROM 'cfb_games_in_data_repo.csv'
WHERE season <> 2002
GROUP BY season
ORDER BY season

-- Miami has not had consitent great seasons in the past 20 years.

,season,wins,loses
0,2003,10.0,2.0
1,2004,8.0,3.0
2,2005,9.0,3.0
3,2006,7.0,6.0
4,2007,5.0,7.0
5,2008,6.0,5.0
6,2009,9.0,4.0
7,2010,7.0,6.0
8,2011,6.0,6.0
9,2012,7.0,5.0


In [6]:
-- Average and Median Wins
WITH totalwins AS (SELECT 
	season,
	sum(CASE WHEN home_team = 'Miami' AND home_points::integer > away_points::integer THEN 1
	   WHEN away_team = 'Miami' AND away_points::integer > home_points::integer THEN 1
	   ELSE 0 end) as wins
FROM 'cfb_games_in_data_repo.csv'
WHERE season <> 2002
GROUP BY season
ORDER BY season)

SELECT 
	round(avg(wins),2) as Average_Wins,
	PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY wins) as Median_wins
FROM totalwins
-- With an average win total of 7.61 and a median of 7, the Hurricanes are not an elite team

,Average_Wins,Median_wins
0,7.61,7.0


In [11]:
-- Lets compare those totals to a couple of the best teams in the past 20 years

--Select Alabamas wins
WITH totalwinsAlabama AS (SELECT 
	season,
	sum(CASE WHEN home_team = 'Alabama' AND home_points::integer > away_points::integer THEN 1
	   WHEN away_team = 'Alabama' AND away_points::integer > home_points::integer THEN 1
	   ELSE 0 end) as wins
FROM 'cfb_games_in_data_repo.csv'
WHERE season <> 2002
		AND home_points <> 'NA'
		AND away_points <> 'NA'
GROUP BY season
ORDER BY season),

--Select Ohio States wins
totalwinsOSU AS (SELECT 
	season,
	sum(CASE WHEN home_team = 'Ohio State' AND home_points::integer > away_points::integer THEN 1
	   WHEN away_team = 'Ohio State' AND away_points::integer > home_points::integer THEN 1
	   ELSE 0 end) as wins
FROM 'cfb_games_in_data_repo.csv'
WHERE season <> 2002
		AND home_points <> 'NA'
		AND away_points <> 'NA'
GROUP BY season
ORDER BY season),

-- Select Oregons wins
totalwinsOregon AS (SELECT 
	season,
	sum(CASE WHEN home_team = 'Oregon' AND home_points::integer > away_points::integer THEN 1
	   WHEN away_team = 'Oregon' AND away_points::integer > home_points::integer THEN 1
	   ELSE 0 end) as wins
FROM 'cfb_games_in_data_repo.csv'
WHERE season <> 2002
		AND home_points <> 'NA'
		AND away_points <> 'NA'
GROUP BY season
ORDER BY season)

--Put them all together
SELECT 
	'Alabama' as team,
	round(avg(wins),2) as Average_Wins,
	PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY wins) as Median_wins
FROM totalwinsAlabama
UNION 
SELECT
	'Ohio State' as team,
	round(avg(wins),2) as Average_Wins,
	PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY wins) as Median_wins
FROM totalwinsOSU
UNION
SELECT
	'Oregon' as team,
	round(avg(wins),2) as Average_Wins,
	PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY wins) as Median_wins
FROM totalwinsOregon


-- When comparing the most consistent teams of the past 20 years, it is clear that Miami has been a step behind them.

,team,Average_Wins,Median_wins
0,Alabama,10.78,12.0
1,Ohio State,10.94,11.0
2,Oregon,9.00,9.0
